In [1]:
import numpy as np
from tqdm import tqdm
import torch

In [2]:
eval_examples = torch.load('../outputs/coqa_examples_train_data.pt')

In [3]:
len(eval_examples)
print(type(eval_examples))

<class 'list'>


### Check if there is a lexical match between the question and the given rational

In [4]:
eval_examples[0]

qas_id: 3zotghdk5ibi9cex97fepx7jetpso7 1, question_text: ['When was the Vat formally opened?'], doc_tokens: [The Vatican Apostolic Library ( ) , more commonly called the Vatican Library or simply the Vat , is the library of the Holy See , located in Vatican City . Formally established in 1475 , although it is much older , it is one of the oldest libraries in the world and contains one of the most significant collections of historical texts . It has 75,000 codices from throughout history , as well as 1.1 million printed books , which include some 8,500 incunabula . The Vatican Library is a research library for history , law , philosophy , science and theology . The Vatican Library is open to anyone who can document their qualifications and research needs . Photocopies for private study of pages from books published between 1801 and 1990 can be requested in person or by mail . In March 2014 , the Vatican Library began an initial four - year project of digitising its collection of manuscr

In [5]:
def removePunctuation(string): 
    """  Removes punctuation marks from
     a given string.
     Reference : https://www.geeksforgeeks.org/removing-punctuations-given-string/
     Eg.
     removePunctuation('Coming for dinner?')
     will return
     'Coming for dinner'
     
     """
    # punctuation marks 
    #punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    punctuations = '''!()[]{};:'"\,<>./?@#$%^&*_~'''
    
  
    # traverse the given string and if any punctuation 
    # marks occur replace it with null 
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "")    
  
    # Print string without punctuation 
    return string 

In [6]:
def check_lexical_match(question_words,rational,words_to_ignore, print_found=False):
    """ Returns True if any of the words in the rational is present wholly 
        or as a prefix in any of the words in question_words, after 
        ignoring words_to_ignore. Else it returns False.
    """
    for word in rational:
        if word.lower() not in words_to_ignore:
            for q in question_words:  
                if(word.lower().startswith(q.lower())): # Does any word in the question form prefix to anyword in the rational (from the passage)?
                    if(print_found):
                        print('{} found'.format(word))
                    return True
                    break
    return False

In [7]:
## trial: checking for lexical match between rational and question

print(len(eval_examples))
eg = eval_examples[1788]
#print(eg.question_text)
start = eg.rational_start_position
end = eg.rational_end_position
print(eg.question_text)
question = eg.question_text[len(eg.question_text)-1]
print(question)
new_question = removePunctuation(question) 
print(new_question)
question_words = new_question.split()
print(question_words)
print(eg.orig_answer_text)
rational = eg.doc_tokens[start:end]
print(rational)
words_to_ignore = ['a','the','on','in','he','she','by','at','for','of','and',
                   'him','her','they','it','to','from']
words_to_ignore = [w.lower() for w in words_to_ignore]
print(check_lexical_match(question_words,rational, words_to_ignore,print_found=True))

108647
['Did he ever stop to learn? No [SEP]', 'Did he have any family support on learning? yes [SEP]', 'Where did he get new ideas from?']
Where did he get new ideas from?
Where did he get new ideas from
['Where', 'did', 'he', 'get', 'new', 'ideas', 'from']
From reading books
['Every', 'time', 'he', 'gets', 'new', 'ideas', 'from', 'the']
gets found
True


In [8]:
lexical_match_labels = np.zeros(len(eval_examples)) # assume no_lexical match to start with
words_to_ignore = ['a','the','on','in','he','she','by','at','for','of','and',
                   'him','her','they','it','to','from']
for i,eg in enumerate(eval_examples):
    question = eg.question_text[len(eg.question_text)-1]
    new_question = removePunctuation(question) 
    question_words = new_question.split()
    rational = eg.doc_tokens[eg.rational_start_position:eg.rational_end_position]
    lexical_match_labels[i] = check_lexical_match(question_words,rational,words_to_ignore)

In [9]:
percentage_lexical_match = sum(lexical_match_labels)/len(lexical_match_labels)*100
print('{:.2f}% of questions have a lexical match with their rationals'.format(percentage_lexical_match))

41.46% of questions have a lexical match with their rationals


In [10]:
np.save('../outputs/labels_of_coqa_lexical_match_train_data.npy',lexical_match_labels) 

In [11]:
lexical_match_labels[2]

0.0

In [12]:
percentage_lexical_match = sum(lexical_match_labels[:25000])/len(lexical_match_labels[:25000])*100
print('{:.2f}% of questions have a lexical match with their rationals'.format(percentage_lexical_match))

40.62% of questions have a lexical match with their rationals
